Trick:  1.  for sending to model always scale by 255
        
        
        2. for cropping use unscalled image otherwise we will get black img in cropped one.
        
        3. if we are reading the images for croppinng so again resize the img same as the input of model

In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)


import os
# comment out below line to enable tensorflow logging outputs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
from absl import app, flags, logging
from absl.flags import FLAGS
# import core.utils as utils
# from core.yolov4 import filter_boxes
from tensorflow.python.saved_model import tag_constants
# from core.config import cfg
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

2.3.0-rc0
2.4.0


In [2]:
# lodding the model
saved_model_loaded = tf.saved_model.load("yolov4-416/", tags=[tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']

In [3]:
img = cv2.imread("images/Inputimage.jpg")

In [4]:
img = cv2.resize(img,(416,416))

In [5]:
img_scaled = img/255

In [6]:
im_height,im_width = img_scaled.shape[:2]
expand = tf.expand_dims(img_scaled,axis=0)
# tf2.x takes only float32 numpy type
expand = tf.cast(expand,dtype="float32")
    
# run image through loaded model
batch_data = tf.constant(expand)
pred_bbox = infer(batch_data)

In [7]:
for key, value in pred_bbox.items():
        boxes = value[:, :, 0:4]
        pred_conf = value[:, :, 4:]

In [8]:
boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
        scores=tf.reshape(
                pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
            max_output_size_per_class=50,
            max_total_size=50,
            iou_threshold=0.50,
        score_threshold=0.75)

In [9]:
# squeeze the dimension
boxes = np.squeeze(boxes)
scores = np.squeeze(scores)
classes = np.squeeze(classes)
num_of_detection = valid_detections.numpy()[0]

In [10]:
# filter out the prediction as per num_of_detection
boxes = boxes[0:int(num_of_detection)]
scores = scores[0:int(num_of_detection)]
classes = classes[0:int(num_of_detection)]

In [11]:
new_boxes = []
for box in boxes:
    ymin = int(box[0]*im_height)
    xmin = int(box[1]*im_width)
    ymax = int(box[2]*im_height)
    xmax = int(box[3]*im_height)
    b = [ymin,xmin,ymax,xmax]
    new_boxes.append(b)

In [12]:
boxes = np.array(new_boxes)

In [13]:
boxes

array([[173, 174, 213, 288]])

In [15]:
for box in boxes:
    img = cv2.imread("images/Inputimage.jpg")
    img = cv2.resize(img,(416,416))
    cropped = img[box[0]:box[2],box[1]:box[3]]
    cv2.imwrite("cropped_new.jpg",cropped)

In [14]:
for box in boxes:
    cropped = img[box[0]:box[2],box[1]:box[3]]
    cv2.imwrite("cropped_new.jpg",cropped)

In [25]:
cv2.imshow("image",img)
cv2.waitKey()
cv2.destroyAllWindows()

In [67]:
cv2.imshow("aa",img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
cv2.rectangle()